In [3]:
from deeppavlov import configs, build_model

ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

In [4]:
ner_model(['Curling World Championship will be held in Antananarivo'])

[[['Curling',
   'World',
   'Championship',
   'will',
   'be',
   'held',
   'in',
   'Antananarivo']],
 [['B-EVENT', 'I-EVENT', 'I-EVENT', 'O', 'O', 'O', 'O', 'B-GPE']]]

In [25]:
import numpy as np
from ml_utils import *
from dialogue import *

In [12]:
text = 'Что будет 11 октября'
tokens, ners = ner_model([text])
tokens = np.array(tokens[0])
ners = np.array(ners[0])
mask = ners != 'O'

In [13]:
tokens

array(['Что', 'будет', '11', 'октября'], dtype='<U7')

In [14]:
ners

array(['O', 'O', 'B-DATE', 'I-DATE'], dtype='<U6')

In [15]:
mask

array([False, False,  True,  True])

In [40]:
models = {'ner_model': ner_model}

In [55]:
import json
import requests
events = requests.get('https://pushkinmuseum.art/json/events.json')
events.json()

{'11421': {'path': 'events/archive/2019/exhibitions/louisvuitton/index.php',
  'm_parent_id': '11417',
  'img_header': {'mob': '/events/archive/2019/exhibitions/louisvuitton/11421_img_mob.jpg',
   'pc': '/events/archive/2019/exhibitions/louisvuitton/11421_img_pc.jpg'},
  'dateBegin': '2019-06-19 00:00:00',
  'dateEnd': '2019-09-29 00:00:00',
  'type': {'ru': 'Выставка', 'en': 'Exhibition'},
  'age': '',
  'ticket': '/#sid=685',
  'name': {'ru': 'Коллекция Fondation Louis Vuitton*. Избранное',
   'en': 'Collection of Fondation Louis Vuitton: Selected Works'},
  'headimg': {'ru': '', 'en': ''},
  'text': {'ru': '<p><a href="https://pushkinmuseum.art/events/archive/2019/exhibitions/louisvuitton/tickets/index.php">Условия покупки билетов и экскурсионных путевок</a></p><p>ГМИИ им. А.С. Пушкина совместно с одним из ведущих мировых собраний современного искусства Fondation Louis Vuitton представляют выставку &laquo;Коллекция Fondation Louis Vuitton. Избранное&raquo;. Зрители увидят знаковые р

In [94]:
from datetime import datetime
from typing import List, Dict

def parse_date(date):
    if date.startswith('0000-00-00'):
        return None
    else:
        return datetime.strptime(date, "%Y-%m-%d %H:%M:%S").date()


def init_events():
    import requests
    # events0 = json.load(open('../../data/events.json'))
    events0 = requests.get('https://pushkinmuseum.art/json/events.json')
    events0 = events0.json()

    events = []
    for event_key, event in events0.items():
        date_begin = parse_date(event['dateBegin'])
        date_end = parse_date(event['dateEnd'])
        if date_begin is None:
            continue
        if date_end is None:
            date_end = date_begin
        events.append((date_begin, date_end, event))
    events.sort(key=lambda event_tuple: -(event_tuple[1] - event_tuple[0]))
    events = [(str(date_begin), str(date_end), event) for date_begin, date_end, event in events]
    return events


events = init_events()


def find_events(date_begin: str, date_end: str) -> List[Dict]:
    def is_date_inside(date, begin, end):
        return begin <= date <= end

    events_filtered = [
        event for event_date_begin, event_date_end, event in events if
        is_date_inside(date_begin, event_date_begin, event_date_end) or
        is_date_inside(date_end, event_date_begin, event_date_end)
    ]
    return events_filtered

In [82]:
text = 'Что будет 11 октября?'
text = 'Что будет в октябре?'

In [83]:
def f(text):
    tokens, ners = models['ner_model']([text])
    tokens = np.array(tokens[0])
    ners = np.array(ners[0])
    mask = ners != 'O'
    lemmatize_text = lemmatize(text.lower())
    text_lemm_set = set(lemmatize_text.split())

    # вопросы вида "Что будет 11 октября?"
    if any('DATE' in ner for ner in ners):
        year = 2019
        month = None
        day = None

        tokens_date = [token for token, ner in zip(tokens, ners) if 'DATE' in ner]

        months = 'январь февраль март апрель май июнь июль август сентябрь октябрь ноябрь декабрь'.split()
        possible_months = text_lemm_set & set(months)
        month = next(iter(possible_months)) if possible_months else None
        if not month:
            return {text: 'todo', meta: ''}
        month = 1 + months.index(month)

        days = set([str(i) for i in range(1, 31 + 1)])
        possible_days = (text_lemm_set | set(tokens)) & set(days)
        day = int(next(iter(possible_days))) if possible_days else None

        date_begin = '{}-{:02}-{:02}'.format(year, month, day if day else 1)
        date_end = '{}-{:02}-{:02}'.format(year, month, day if day else 31)

        events = find_events(date_begin, date_end)
        events = [event for event in events if 'ru' in event['name']]
        events = events[:3]
        text = '.\n\n'.join([event['name']['ru'] for event in events]) + '.'
        print(text)

f(text)

Концерты в Мемориальной квартире Святослава Рихтера.

«Русский Йорданс». Картины и рисунки Якоба Йорданса из собраний России.

Восточный джаз / East West Jazz.


In [75]:
#NAVIGATION 
navigation_words = 'где находится туалет кафе камера хранения пройти карта столовая куда пойти идем зал найти путь'
navigation_phrases = ['в каком зале', 'где висит', 'где стоит', 'где найти', 'где выставляется', 'где посмотреть', 'как пройти', 'какой маршрут']

#CULTURE
culture_words = 'выставка картины скульптуры расскажи какой технике найди аудио дорожка скачать список покажи книга книги о работа работах жанр годы друзья список изданий каталог почему покажите знаменитые расскажи кто страна РОССИЯ ЕВРОПА регион государство место город местечно край'
culture_phrases = ['найти что-то', 'что есть', 'список изданий', 'что посмотреть', 'покажите предметы', 'у вас есть', 'знаменитые картины', 'откуда', 'есть ли','расскажи о', 'создатель', 'художник', 'скульптор']

#OTHER
other_words = 'когда будет дата январь февраль март апрель май июнь июль август сентябрь октябрь ноябрь декабрь'
other_phrases = ['когда будет', 'дата проведения', 'когда проводится']

def make_find_set(words, phrases):
    lemmed_words = lemmatize(words.lower()).split()
    phrases = [phrase.lower() for phrase in phrases]
    find_set = set(lemmed_words + phrases)
    return find_set


find_sets = [make_find_set(navigation_words, navigation_phrases), 
             make_find_set(culture_words, culture_phrases), 
             make_find_set(other_words, other_phrases)]


def classify(text, find_sets):
    lemmed = lemmatize(text.lower())
    finded = np.zeros(len(find_sets))
    for i, find_set in enumerate(find_sets):
        for item in find_set:
            if item in lemmed:
                finded[i] += 1
    result_class = np.argmax(finded)
    return result_class

In [76]:
classify('что будет 11 октября', find_sets)

2